In [1]:
import random as rand;
import copy as cpy;

In [2]:
class GameTile:
  def __init__(self):
    self.tile_type = " ";
    self.player = 0;
    #for agent
    self.tile_value = 0;

class Board:
  def __init__(self):
    self.board = [[GameTile() for i in range(4)] for j in range(4)];

  def winning_moves(self, player):
    #return a list of winning moves
    pass;
  def merit_moves(self, player):
    #return a list of merit moves
    pass;

  def check_move(self):
    for w in self.get_runs(4):
        return True;
    return False;

  def get_runs(self, run_size):
    #https://stackoverflow.com/questions/3311119/determining-three-in-a-row-in-python-2d-array
    # Offsets to find the previous cell in all four directions.
    offsets = {'h' : ( 0, -1), 'v' : (-1,  0), 'f' : (-1,  1), 'b' : (-1, -1)}

    # Helpers to check for valid array bounds and to return a new cell dict.
    size = len(self.board)
    in_bounds = lambda r, c: r >= 0 and c >= 0 and r < size and c < size
    new_cell = lambda i, j, p: dict(h=1, v=1, f=1, b=1, i=i, j=j, player=p)

    # Use the raw grid to create a grid of cell dicts.
    grid = []
    for i, row in enumerate(self.board):
        grid.append([])
        for j, tile in enumerate(row):
            # Add a cell dict to the grid (or None for empty spots).
            cell = new_cell(i, j, tile.player) if tile.player else None
            grid[i].append(cell)
            if not cell: continue

            # For each direction, look to the previous cell. If it matches the
            # current player, we can extend the run in that direction.
            for d, offset in offsets.items():
                r, c = (i + offset[0], j + offset[1])
                if in_bounds(r, c):
                    prev = grid[r][c]
                    if prev and prev['player'] == cell['player']:
                        # We have a match, so the run size is one bigger,
                        # and we will track that run in the current cell,
                        # not the previous one.
                        cell[d] = prev[d] + 1
                        prev[d] = None

    # For all non-None cells, yield run info for any runs that are big enough.
    for cell in (c for row in grid for c in row if c):
        for d in offsets:
            if cell[d] and cell[d] >= run_size:
                yield dict(player = cell['player'], endpoint = (cell['i'], cell['j']), direction = d, run_size = cell[d])
  
  def full(self):
    for tile_list in self.board:
      for tile in tile_list:
        if tile.player == 0:
          return False;

    return True;

  def available(self):
    av = 0;

    for tile_list in self.board:
      for tile in tile_list:
        if tile.player == 0:
          av += 1;

    return av;

  def check_merit(self):
    score = [0, 0]
    for merit_move in self.get_runs(3):
      if merit_move['player'] == PlayerCategory.AGENT:
        score[0] += 1;
      elif merit_move['player'] == PlayerCategory.HUMAN:
        score[1] += 1;
    
    return (score[0], score[1]);

  def print(self):
    row = 0;
    for tile_list in self.board:
      col = 0;
      for tile in tile_list:
        print(tile.tile_type, end='');
        if col in [0, 1, 2]:
          print("|", end='');
        col += 1;
      print();
      if row in [0, 1, 2]:
        print("-------");
      row += 1;

In [3]:
class PlayerCategory:
  AGENT = 1;
  HUMAN = 2;

class Human:
  def move(self, game_board):
    size = len(game_board.board)
    in_bounds = lambda r, c: r >= 0 and c >= 0 and r < size and c < size

    top_offsets = {'h' : ( 0, -1), 'v' : (-1,  0), 'f' : (-1,  1), 'b' : (-1, -1)}
    bot_offsets = {'h' : ( 0, 1), 'v' : (1,  0), 'f' : (1,  -1), 'b' : (1, 1)}

    #iterate through each possible move and just make a temp board and check which move give the highest value play
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";
          
          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            tile.player = PlayerCategory.HUMAN;
            tile.tile_type = "o";
            return;

    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if enemy can win
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";

          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            tile.player = PlayerCategory.HUMAN;
            tile.tile_type = "o";
            return;


    best_move = [0, 0, 0];
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check for best merit play
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > best_move[0]:
            best_move[0] = len(winning_move_list);
            best_move[1] = i;
            best_move[2] = j;

    if best_move[0] > 0:
      game_board.board[best_move[1]][best_move[2]].player = PlayerCategory.HUMAN;
      game_board.board[best_move[1]][best_move[2]].tile_type = "o";
      return;
    
    best_move = [0, 0, 0];
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check for best merit block
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > best_move[0]:
            best_move[0] = len(winning_move_list);
            best_move[1] = i;
            best_move[2] = j;

    if best_move[0] > 0:
      game_board.board[best_move[1]][best_move[2]].player = PlayerCategory.HUMAN;
      game_board.board[best_move[1]][best_move[2]].tile_type = "o";
      return;

    #random move
    r, c = (rand.randint(0, 3), rand.randint(0, 3));
    while game_board.board[r][c].player != 0:
      r, c = (rand.randint(0, 3), rand.randint(0, 3));
    game_board.board[r][c].player = PlayerCategory.HUMAN;
    game_board.board[r][c].tile_type = "o";

class Agent:
  def __init__(self):
    self.markov_board = [[0 for i in range(4)] for j in range(4)];

  def reset_markov(self):
    self.markov_board = [[0 for i in range(4)] for j in range(4)];

  def exit_rewards(self, game_board):
    #check for possible rows of 2 (2)
    #iterate through each possible move and just make a temp board and check which move give the highest value play
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(2):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            self.markov_board[i][j] = 2 * len(winning_move_list);
    #check for possible rows of 3 (6)
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            self.markov_board[i][j] = 30 * len(winning_move_list);
    #check for possible enemy rows of 3 (3)
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            self.markov_board[i][j] = 15 * len(winning_move_list);
    #check for possible rows of 4 (60)
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            self.markov_board[i][j] = 1000 * len(winning_move_list);
    #check for possible enemy rows of 4 (30)
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";
          
          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            self.markov_board[i][j] = 500 * len(winning_move_list);
    
  def summation(self, probability, r, c, board):
    sum = 0;

    for i, tile_list in enumerate(board):
      for j, tile in enumerate(tile_list):
        if i != r and j != c:
          sum += probability * tile;

    return sum;

  def it(self, gamma, epsilon, game_board):
    sum = 0;
    new_sum = 0;
    for i, tile_list in enumerate(self.markov_board):
      for j, tile in enumerate(tile_list):
        if game_board.board[i][j].player == 0:
          sum += tile;
          tile = tile + gamma * self.summation((1/game_board.available()), i, j, self.markov_board);
          new_sum += tile;

    if not new_sum == 0 and sum / new_sum < epsilon:
      self.it(gamma, epsilon, game_board);
    
    return;

  #Task 1
  def markov_populate(self, gamma, epsilon, game_board):
    self.reset_markov();
    self.exit_rewards(game_board);
    #run the markov to add living rewards
    self.it(gamma, epsilon, game_board);
    # print(self.markov_board);
    
  def move(self, gamma, epsilon, game_board, print_flag = False):
    #markov_populate
    self.markov_populate(gamma, epsilon, game_board);
    #decide on move
    best_move = [0, 0, 0];
    for i, tile_list in enumerate(self.markov_board):
      for j, tile in enumerate(tile_list):
        if tile > best_move[0]:
          best_move[0] = tile;
          best_move[1] = i;
          best_move[2] = j;

    if best_move[0] > 0:
      # print(best_move);
      game_board.board[best_move[1]][best_move[2]].player = PlayerCategory.AGENT;
      game_board.board[best_move[1]][best_move[2]].tile_type = "x";
      if print_flag and game_board.check_move():
        for row, tile_list in enumerate(self.markov_board):
          for col, tile in enumerate(tile_list):
            print(tile, end='');
            if col in [0, 1, 2]:
              print("|", end='');
          print();
          if row in [0, 1, 2]:
            print("-------");
    else:
      #random move
      r, c = (rand.randint(0, 3), rand.randint(0, 3));
      while game_board.board[r][c].player != 0:
        r, c = (rand.randint(0, 3), rand.randint(0, 3));
      game_board.board[r][c].player = PlayerCategory.AGENT;
      game_board.board[r][c].tile_type = "x";

  def human_move(self, game_board):
    size = len(game_board.board)
    in_bounds = lambda r, c: r >= 0 and c >= 0 and r < size and c < size

    top_offsets = {'h' : ( 0, -1), 'v' : (-1,  0), 'f' : (-1,  1), 'b' : (-1, -1)}
    bot_offsets = {'h' : ( 0, 1), 'v' : (1,  0), 'f' : (1,  -1), 'b' : (1, 1)}

    #iterate through each possible move and just make a temp board and check which move give the highest value play
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if can win
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            tile.player = PlayerCategory.AGENT;
            tile.tile_type = "x";
            return;

    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check if enemy can win
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";

          for winning_move in temp_board.get_runs(4):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > 0:
            tile.player = PlayerCategory.AGENT;
            tile.tile_type = "x";
            return;


    best_move = [0, 0, 0];
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check for best merit play
          temp_board.board[i][j].player = PlayerCategory.AGENT;
          temp_board.board[i][j].tile_type = "x";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > best_move[0]:
            best_move[0] = len(winning_move_list);
            best_move[1] = i;
            best_move[2] = j;

    if best_move[0] > 0:
      game_board.board[best_move[1]][best_move[2]].player = PlayerCategory.AGENT;
      game_board.board[best_move[1]][best_move[2]].tile_type = "x";
      return;
    
    best_move = [0, 0, 0];
    for i, tile_list in enumerate(game_board.board):
      for j, tile in enumerate(tile_list):
        temp_board = Board();
        temp_board.board = cpy.deepcopy(game_board.board);
        if tile.player == 0:
          winning_move_list = [];
          #check for best merit block
          temp_board.board[i][j].player = PlayerCategory.HUMAN;
          temp_board.board[i][j].tile_type = "o";
          
          for winning_move in temp_board.get_runs(3):
            winning_move_list.append(winning_move);

          if len(winning_move_list) > best_move[0]:
            best_move[0] = len(winning_move_list);
            best_move[1] = i;
            best_move[2] = j;

    if best_move[0] > 0:
      game_board.board[best_move[1]][best_move[2]].player = PlayerCategory.AGENT;
      game_board.board[best_move[1]][best_move[2]].tile_type = "x";
      return;

    #random move
    r, c = (rand.randint(0, 3), rand.randint(0, 3));
    while game_board.board[r][c].player != 0:
      r, c = (rand.randint(0, 3), rand.randint(0, 3));
    game_board.board[r][c].player = PlayerCategory.AGENT;
    game_board.board[r][c].tile_type = "x";

In [4]:
agent = Agent();
human = Human();

#TASK 2
#win/merit/loss ratio
scores = {'agent': [0, 0, 0], 'human': [0, 0, 0]}

gamma, epsilon = (.3, .6);
for i in range(100):
  game = Board();
  #game loop
  while True:
    #player move
    # print("agent move");
    agent.move(gamma, epsilon, game);
    # game.print();
    if game.check_move():
      # print("agent won");
      scores['agent'][0] += 1;
      scores['human'][2] += 1;
      break;

    #player move
    # print("human move")
    human.move(game);
    # game.print();
    if game.check_move():
      # print("human won");
      scores['agent'][2] += 1;
      scores['human'][0] += 1;
      break;

    #check if game is full
    if game.full():
      agent_score, human_score = game.check_merit();
      if agent_score > human_score:
        # print("agent won");
        scores['agent'][1] += 1;
        scores['human'][2] += 1;
        break;
      elif human_score > agent_score:
        # print("human won");
        scores['agent'][2] += 1;
        scores['human'][1] += 1;
        break;
      # print("it's a tie");
      scores['agent'][2] += 1;
      scores['human'][2] += 1;
      break;

print(scores);

#TASK 3
#win/merit/loss ratio
scores = {'agent': [0, 0, 0], 'human': [0, 0, 0]}

gamma, epsilon = (.9, .575);
for i in range(100):
  game = Board();
  #game loop
  while True:
    #player move
    # print("agent move");
    agent.move(gamma, epsilon, game);
    # game.print();
    if game.check_move():
      # print("agent won");
      scores['agent'][0] += 1;
      scores['human'][2] += 1;
      break;

    #player move
    # print("human move")
    human.move(game);
    # game.print();
    if game.check_move():
      # print("human won");
      scores['agent'][2] += 1;
      scores['human'][0] += 1;
      break;

    #check if game is full
    if game.full():
      agent_score, human_score = game.check_merit();
      if agent_score > human_score:
        # print("agent won");
        scores['agent'][1] += 1;
        scores['human'][2] += 1;
        break;
      elif human_score > agent_score:
        # print("human won");
        scores['agent'][2] += 1;
        scores['human'][1] += 1;
        break;
      # print("it's a tie");
      scores['agent'][2] += 1;
      scores['human'][2] += 1;
      break;

print(scores);


{'agent': [4, 62, 34], 'human': [3, 14, 83]}
{'agent': [2, 48, 50], 'human': [5, 15, 80]}


In [5]:
gamma, epsilon = (.4, .6);
end_flag = False;
for i in range(100):
  game = Board();
  #game loop
  while True:
    #player move
    # print("agent move");
    agent.move(gamma, epsilon, game, True);
    # game.print();
    if game.check_move():
      # print("agent won");
      game.print();
      end_flag = True;
      break;

    #player move
    # print("human move")
    human.move(game);
    # game.print();
    if game.check_move():
      # print("human won");
      break;

    #check if game is full
    if game.full():
      agent_score, human_score = game.check_merit();
      if agent_score > human_score:
        # print("agent won");
        break;
      elif human_score > agent_score:
        # print("human won");
        break;
      # print("it's a tie");
      break;
    
    if end_flag:
      break;

0|0|0|0
-------
0|0|0|0
-------
0|0|0|0
-------
0|1000|60|0
o|x|o|x
-------
x|x|x|o
-------
o|x|x|o
-------
o|x| |o
